In [ ]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
from os.path import join
from tqdm import tqdm
import pandas as pd
from os.path import expanduser
import sys
from typing import List
import numpy as np
import joblib
from pprint import pprint
import imodelsx.util
from os.path import dirname
import pickle as pkl
import json
from copy import deepcopy
from numpy.linalg import norm
from math import ceil
# from imodelsx.qaemb.qaemb import QAEmb, get_sample_questions_and_examples
from neuro.ecog.config import STORIES_POPULAR, STORIES_UNPOPULAR, ECOG_DIR

# Look at question answers

In [ ]:
suffix_qs = '___qs_35_stable'

# save ensemble feats
# settings = ['sec_1.5']
settings = ['sec_3']
# settings = ['words']
# settings = ['sec_6']
ensemble1 = [
    # 'mistralai/Mistral-7B-Instruct-v0.2',
    # 'meta-llama/Meta-Llama-3-8B-Instruct',
    # 'google/gemma-7b-it',
    'gpt-4o-mini',
]
# story_fnames = ['ant-man.pkl']
# story_fnames = ['lotr-1.pkl']
story_fnames = ['___podcasts-story___.pkl']

for setting in settings:
    print(setting)

    # read in ensemble feats
    ensemble_checkpoint_story_dict = {}
    for checkpoint in tqdm(ensemble1):
        checkpoint_clean = checkpoint.replace('/', '___')
        output_dir_clean = join(ECOG_DIR, f'features{suffix_qs}',
                                checkpoint_clean, setting)
        checkpoint_story_dict = {}
        for story_fname in story_fnames:
            if story_fname.endswith('.pkl'):
                checkpoint_story_dict[story_fname] = joblib.load(
                    join(output_dir_clean, story_fname))
        ensemble_checkpoint_story_dict[checkpoint] = deepcopy(
            checkpoint_story_dict)

In [ ]:
easy_qs = ['Is time mentioned in the input?',
           'Does the input contain a measurement?', 'Does the input contain a number?']
for checkpoint in ensemble1:
    print(checkpoint)
    # [easy_qs]
    df = ensemble_checkpoint_story_dict[checkpoint][story_fnames[0]]
    for k in df.columns:
        print(k, df[k][df[k] > 0].index.tolist())
    print(df.shape)

In [ ]:
df.head(500)

In [ ]:
df_audio = joblib.load(join(
    ECOG_DIR, 'features_audio/gpt-4o-audio-preview/sec_3', '___podcasts-story___.pkl'))

# df = df_word.merge(df_audio, on='word_idx', how='left')
# /home/chansingh/mntv1/ecog/features_audio/gpt-4o-audio-preview/sec_3

In [ ]:
!rclone copy /home/chansingh/mntv1/ecog/features_audio/gpt-4o-audio-preview/ box:DeepTune/QA/ecog_cached_qa_features/gpt-4o-audio-preview --progress